## 1.3 Formulating Abstractions with Higher-Order Procedures

### 1.3.1 Procedures as Arguments

In [115]:
(define (sum term a next b)
  (if (> a b)
    0
    (+ (term a)
       (sum term (next a) next b))))

In [3]:
(define (cube n) (* n n n))
(define (sum-cubes a b)
  (define (inc n) (+ n 1))
  (sum cube a inc b))

In [4]:
(sum-cubes 1 10)

3025

In [5]:
(define (sum-integers a b)
  (define (identity n) n)
  (define (inc n) (+ n 1))
  (sum identity a inc b))

In [6]:
(sum-integers 1 10)

55

In [7]:
(define (pi-sum a b)
  (define (pi-term x)
    (/ 1.0 (* x (+ x 2))))
  (define (pi-next x)
    (+ x 4))
  (sum pi-term a pi-next b))

In [8]:
(* 8 (pi-sum 1 1000))

3.139592655589783

In [9]:
(define (integral f a b dx)
  (define (add-dx x)
    (+ x dx))
  (* (sum f (+ a (/ dx 2.0)) add-dx b)
     dx))

In [10]:
(integral cube 0 1 0.01)

0.24998750000000042

In [11]:
(integral cube 0 1 0.001)

0.249999875000001

#### Exercise 1.29: Simpson’s Rule

- integral of f between a and b 
  = (h/3) (y_0 + 4y_1 + 2y_2 + 4y_3 + ... + 2y_n-2 + 4y_n-1 + y_n)
- h = (b - a)/n
- given integer `n` (Larger `n` increases the accuracy of proximation)
- y_k = f(a + kh)

In [43]:
(define (simpson-integral f a b n)
  (define (next x)
    (+ x 1))
  (define h
    (/ (- b a) n))
  (define (term k)
    (define (y k)
      (f (+ a (* k h))))
    (cond
      ((or (= k 0) (= k n)) (y k))
      ((even? k) (* 4 (y k)))
      (else (* 2 (y k)))))
  ; (* (/ h 3) (sum term a next n)) used `3.0` to get result in decimal,            not fraction
  (* (/ h 3.0) (sum term a next n))
  )

In [45]:
(display (format "~a\n" (simpson-integral cube 0 1 100)))
(display (format "~a\n" (simpson-integral cube 0 1 1000)))

0.2467166666666667
0.24966716666666663


Due to `integral of cube between 0 and 1` is 0.25, result of `(simpson-integral cube 0 1 1000)` has better accuracy.

#### Exercise 1.30: iterative `sum` procedure

```s
(define (sum term a next b)
  (define (iter a result)
    (if (?)
        ?
        (iter ? ?)))
  (iter ? ?))
```

In [53]:
(define (sum-iter term a next b)
  (define (iter a result)
    (if (> a b)
        result
        (iter (next a) (+ (term a) result))))
  (iter a b))

In [56]:
(define (sum-cubes a b)
  (define (cube n) (* n n n))
  (define (inc n) (+ n 1))
  (sum-iter cube a inc b))
(sum-cubes 1 10)

3035

#### Exercise 1.31: product

- a. implement it and compute factorial and approximations of pi using it

In [46]:
(define (product term a next b)
  (if (> a b)
      1
      (*
        (term a)
        (product term (next a) next b))))

In [58]:
(define (factorial a b)
  (define (next x) (+ x 1))
  (define (term a) a)
  (product term a next b))

In [60]:
(display (format "~a\n" (factorial 1 5)))
(display (format "~a\n" (factorial 1 10)))

120
3628800


1, 2, 3, 4, 5, 6, ...

dividend

```
2, 4, 4, 6, 6, 8 ...
-----
1, 2, 3, 4, 5 ...

=> n / 2
0.5, 1, 1.5, 2.0, 2.5 ...

=> + 0.5
1, 1.5, 2.0, 2.5, 3.0 ...

=> even: + 0.5, odd: + 0
1, 2, 2, 3, 3 ...

=> * 2
2, 4, 4, 6, 6, 8 ...
```

divisor
```
3, 3, 5, 5, 7, 7, ...
----
1, 2, 3, 4, 5 ...

=> n / 2
0.5, 1, 1.5, 2.0, 2.5 ...

=> even: + 0, odd: + 0.5
1, 1, 2, 2, 3 ...

=> * 2
2, 2, 4, 4, 6, ...

=> + 1
3, 3, 5, 5, 7 ...

```

In [32]:
(define (approx-pi depth)
  ; (define (product term a next b)
  ;   (display (format "~a, ~a\n" a (term a)))
  ;   (if (> a b)
  ;       1
  ;       (*
  ;         (term a)
  ;         (product term (next a) next b))))
  (define (next x) (+ x 1))
  (define (dividend)
    (define (term x) 
      (* 2 (+ (/ x 2) (if (even? x) 1 0.5))))
    ; (newline)
    (product term 1.0 next depth))
  (define (divisor)
    (define (term x) 
      (+ (* (+ (/ x 2) (if (even? x) 0 0.5)) 2) 1))
    ; (newline)
    (product term 1.0 next depth))
  (* 4 (/ (dividend) (divisor))))

In [29]:
(approx-pi 5)


1.0, 2.0
2.0, 4.0
3.0, 4.0
4.0, 6.0
5.0, 6.0
6.0, 8.0

1.0, 3.0
2.0, 3.0
3.0, 5.0
4.0, 5.0
5.0, 7.0
6.0, 7.0


2.9257142857142857

In [35]:
(display (format "~a\n" (approx-pi 100)))
(display (format "~a\n" (approx-pi 200)))

3.1570301764551667
nan


By doing division after computing both divisor and diviend, integer had been overflowen

In [40]:
(define (approx-pi depth)
  (define (next x) (+ x 1))
  (define (term x) 
    (/
      (* 2 (+ (/ x 2) (if (even? x) 1 0.5)))
      (+ (* (+ (/ x 2) (if (even? x) 0 0.5)) 2) 1)))
  (* 4 (product term 1 next depth)))

In [43]:
(display (format "~a\n" (approx-pi 5)))
(display (format "~a\n" (approx-pi 10)))
(display (format "~a\n" (approx-pi 100)))
(display (format "~a\n" (approx-pi 200)))
(display (format "~a\n" (approx-pi 1000)))

2.9257142857142853
3.275101041334807
3.1570301764551645
3.149378473168593
3.143160705532257


- b. implement with method (recuresive or iterative) opposite of a

In [49]:
(define (product-iter term a next b)
  (define (iter a acc)
    (if (> a b)
        acc
        (iter (next a) (* acc (term a)))))
  (iter a 1))

In [50]:
(define (factorial-iter a b)
  (define (next x) (+ x 1))
  (define (term a) a)
  (product-iter term a next b))

In [52]:
(display (format "~a\n" (factorial 1 5)))
(display (format "~a\n" (factorial 1 10)))

120
3628800


#### Exercise 1.32: accumulate

- a. implement `accumulate`

In [53]:
(define (accumulate combiner null-value term a next b)
  (if (> a b)
      null-value
      (combiner
        (term a)
        (accumulate combiner null-value term (next a) next b))))

In [56]:
(define (acc-sum a b)
  (define (combiner x y) (+ x y))
  (define (term x) x)
  (define (next x) (+ x 1))
  (accumulate combiner 0 term a next b))

In [58]:
(display (format "~a\n" (acc-sum 1 10)))
(display (format "~a\n" (acc-sum 1 100)))

55
5050


In [59]:
(define (acc-product a b)
  (define (combiner x y) (* x y))
  (define (term x) x)
  (define (next x) (+ x 1))
  (accumulate combiner 1 term a next b))

In [61]:
(display (format "~a\n" (acc-product 1 5)))
(display (format "~a\n" (acc-product 1 10)))

120
3628800


- b. implement `accumulate` using opposite method (recuresive or iterative) of a

In [62]:
(define (accumulate-iter combiner null-value term a next b)
  (define (iter k result)
    (if (> k b)
        result
        (iter (next k) (combiner (term k) result))))
  (iter a null-value)
)

In [67]:
(define (acc-iter-sum a b)
  (define (combiner x y) (+ x y))
  (define (term x) x)
  (define (next x) (+ x 1))
  (accumulate-iter combiner 0 term a next b))

In [68]:
(display (format "~a\n" (acc-iter-sum 1 10)))
(display (format "~a\n" (acc-iter-sum 1 100)))

55
5050


#### Exercise 1.33: filtered-accumulate

In [97]:
(define (filtered-accumulate combiner null-value term a next b filter)
  ; (display (format "~a: ~a\n" a (filter a)))
  (cond 
      ((> a b) null-value)
      ((filter a)
        (combiner
          (term a)
          (filtered-accumulate combiner null-value term (next a) next b filter)))
      (else (filtered-accumulate combiner null-value term (next a) next b filter))
  ))

In [74]:
(define (sum-even a b)
  (define (combiner x y) (+ x y))
  (define (term x) x)
  (define (next x) (+ x 1))
  (define (filter x) (even? x))
  (filtered-accumulate combiner 0 term a next b filter))

In [77]:
(display (format "~a\n" (+ 2 4 6)))
(display (format "~a\n" (sum-even 1 6)))

12
12


- a. the sum of the squares of the prime numbers in the interval a to b

In [84]:
; from ss 1.2
(define (prime? n)
  (define (smallest-divisor n)
    (define (square n) (* n n))
    (define (divides? a b) (= (remainder b a) 0))
    (define (find-divisor n test)
      (cond
        ((> (square test) n) n)
        ((divides? test n) test)
        (else (find-divisor n (+ test 1)))))
    (find-divisor n 2))
  (= n (smallest-divisor n)))

In [85]:
(define (sum-suquared-prime a b)
  (define (combiner x y) (+ x y))
  (define (term x) (* x x))
  (define (next x) (+ x 1))
  (define (filter x) (prime? x))
  (filtered-accumulate combiner 0 term a next b filter))

In [86]:
(display (format "~a\n" (+ 1 4 9 25)))
(display (format "~a\n" (sum-suquared-prime 1 6)))

39
39


- b. the product of all the positive integers less than `n` that are relatively prime to `n`

In [87]:
; from ss 1.2
(define (gcd a b)
  (if (= b 0)
    a
    (gcd b (remainder a b))))

In [107]:
(define (product-smaller-rel-prime-to n)
  (define (combiner x y) (* x y))
  (define (null-value) 1)
  (define (term x) x)
  (define (next x) (+ x 1))
  (define (filter x) (= 1 (gcd x n)))
  (filtered-accumulate combiner (null-value) term 1 next n filter))

In [108]:
(display (format "~a\n" (* 1 3 5 7)))
(display (format "~a\n" (product-smaller-rel-prime-to 8)))

105
105


### 1.3.2 Constructing Procedures Using lambda

```s
(lambda (⟨formal-parameters⟩) ⟨body⟩)
```

In [116]:
(define (pi-sum a b)
  (sum (lambda (x) (/ 1.0 (* x (+ x 2))))
        a
        (lambda (x) (+ x 4))
        b))

In [117]:
(* 8 (pi-sum 1 1000))

3.139592655589783

#### Using `let` to create local variables

`let` definition

```s
(let ((⟨var_1⟩ ⟨exp_1⟩)
      (⟨var_2⟩ ⟨exp_2⟩)
      ...
      (⟨var_n⟩ ⟨exp_n⟩)) 
    ⟨body⟩)
```

`let` is syntax sugar of `lambda`. Lisp iterprets `let` like this.
w
```s
((lambda (⟨var_1⟩ ... ⟨var_n⟩)
    ⟨body⟩)
  ⟨exp_1⟩
  ⟨exp_2⟩)
```

In [118]:
(let ((a 1)
      (b 2))
  (+ a b))

3

#### Exercise 1.34: apply `(f f)`

~cause infinite loop~

In [119]:
(define (f g) (g 2))

In [120]:
(f f)


Traceback (most recent call last):
  File "In [120]", line 1, col 1, in 'f'
  File "In [119]", line 1, col 15, in 'g'
  File "In [119]", line 1, col 15, in 'g'
  File "In [119]", line 1, col 15
RunTimeError: attempt to apply non-procedure '2'



Lisp evaluates procedure with `applicative-order evaluation`. Then, `(f f)` will be evaluated steps like this

```s
(f f)
=> (g 2)
=> (f 2)
=> (g 2)
=> (2 2)
```

### 1.3.3 Procedures as General Methods

#### Finding roots of equations by the half-interval method

In [147]:
(define (average x y) (/ (+ x y) 2))
(define (positive? x) (or (> x 0) (= x 0)))
(define (negative? x) (< x 0))
(define (search f neg-point pos-point)
  (let ((midpoint (average neg-point pos-point)))
    (if (close-enough? neg-point pos-point)
        midpoint
        (let ((test-value (f midpoint)))
          (cond
            ((positive? test-value) (search f neg-point midpoint))
            ((negative? test-value) (search f midpoint pos-point))
            (else midpoint))))))

In [148]:
(define (close-enough? x y) (< (abs (- x y)) 0.001))

In [159]:
(search
  (lambda (x) (/ x 3))
  -1.0
  1.0)

-0.00048828125

In [153]:
(search
  (lambda (x) (* x x x))
  -1.0
  1.0)

-0.00048828125

In [172]:
(define (half-interval-method f a b)
  (let ((a-val (f a))
        (b-val (f b)))
    (cond
      ((and (negative? a-val) (positive? b-val)) (search f a b))
      ((and (negative? b-val) (positive? a-val)) (search f b a))
      (else (error "Values are not of opposite sign" a b)))))

In [173]:
(half-interval-method 
  (lambda (x) (- (* x x x) (* 2 x) 3))
  1.0
  2.0)

1.89306640625

In [167]:
((lambda (x) (- (* x x x) (* 2 x) 3)) 1.89306640625)

-0.0019499402260407805

#### Finding fixed points of functions

In [3]:
(define tolerance 0.00001)
(define (fixed-point f first-guess)
  (define (close-enough? v1 v2)
    (< (abs (- v1 v2))
       tolerance))
  ; (define (try guess) <- can't overwride `try`?
  (define (try-it guess)
    (display (format "guess: ~a\n" guess))
    (let ((next (f guess)))
      (if (close-enough? guess next)
          guess
          (try-it next))))
  (try-it first-guess))

In [206]:
(fixed-point cos 1.0)


Traceback (most recent call last):
  File "In [206]", line 1, col 14
RunTimeError: unbound variable 'cos'



In [207]:
(define (sqrt x)
  (fixed-point (lambda (y) (/ x y)) 1.0))

In [208]:
(sqrt 4)

first-guess: 1.0


1.0

In [229]:
(define (new-sqrt x)
  (fixed-point 
    (lambda (y) (average y (/ x y))) 
    1.0))

In [230]:
(new-sqrt 9)

guess: 1.0
guess: 5.0
guess: 3.4
guess: 3.023529411764706
guess: 3.00009155413138
guess: 3.000000001396984


3.000000001396984

#### Exercise 1.35: golden ratio

In [231]:
(define (golden-ratio)
  (fixed-point 
    (lambda (x) (+ 1 (/ 1 x))) 
    1.0))

In [232]:
(golden-ratio)

guess: 1.0
guess: 2.0
guess: 1.5
guess: 1.6666666666666665
guess: 1.6
guess: 1.625
guess: 1.6153846153846154
guess: 1.619047619047619
guess: 1.6176470588235294
guess: 1.6181818181818182
guess: 1.6179775280898876
guess: 1.6180555555555556
guess: 1.6180257510729614
guess: 1.6180371352785146


1.6180371352785146

#### Exercise 1.36: Modify `fixed-point`

In [234]:
; from ss 1.2
(define (fast-exp b n)
  (define (square n) (* n n))
  (define (exp b n)
    (cond 
      ((= n 0) 1)
      ((even? n) (square (exp b (/ n 2))))
      (else (* b (exp b (- n 1))))))
  (exp b n))

In [241]:
(import "math")

(math.log 100 10)

2.0

In [250]:
(import "math")

(define (x_exp_x)
  (fixed-point 
    (lambda (x) (/ (math.log 1000) (math.log x))) 
    2.0))

In [244]:
; (Note that you cannot start fixed-point with a guess of 1, as this would cause division by log(1) = 0.)
(math.log 1.0)

0.0

In [251]:
(x_exp_x)

guess: 2.0
guess: 9.965784284662087
guess: 3.004472209841214
guess: 6.279195757507157
guess: 3.759850702401539
guess: 5.215843784925895
guess: 4.182207192401397
guess: 4.8277650983445906
guess: 4.387593384662677
guess: 4.671250085763899
guess: 4.481403616895052
guess: 4.6053657460929
guess: 4.5230849678718865
guess: 4.577114682047341
guess: 4.541382480151454
guess: 4.564903245230833
guess: 4.549372679303342
guess: 4.559606491913287
guess: 4.552853875788271
guess: 4.557305529748263
guess: 4.554369064436181
guess: 4.556305311532999
guess: 4.555028263573554
guess: 4.555870396702851
guess: 4.555315001192079
guess: 4.5556812635433275
guess: 4.555439715736846
guess: 4.555599009998291
guess: 4.555493957531389
guess: 4.555563237292884
guess: 4.555517548417651
guess: 4.555547679306398
guess: 4.555527808516254
guess: 4.555540912917957


4.555540912917957

4.555540912917957 ^ 4.555540912917957 = 1,000.0131045064

#### Exercise 1.37: An infinite continued fraction

In [54]:
(define (cont-frac n d k)
  (define (iter i)
    ; (display (format "k: ~a\n" k))
    (if (= k i)
      (d i)
      (/ (n i) (+ (d i) (iter (+ i 1))))))
  (display (format "depth: ~a\n" k))
  (iter 1))

In [55]:
(cont-frac
  (lambda (i) 1.0)
  (lambda (i) 1.0)
  3)

depth: 3


0.6666666666666666

In [57]:
(display (format "~a\n" (cont-frac (lambda (i) 1.0) (lambda (i) 1.0) 10)))
(display (format "~a\n" (cont-frac (lambda (i) 1.0) (lambda (i) 1.0) 100)))

depth: 10
0.6179775280898876
depth: 100
0.6180339887498948


b. iterative process

In [62]:
(define (cont-frac-iter n d k)
  (define (iter i acc)
    ; (display (format "k: ~a, acc: ~a\n" k acc))
    (if (= k i)
        acc
        (iter (+ i 1) (/ (n i) (+ (d i) acc)))))
  (display (format "depth: ~a\n" k))
  (iter 1 1))

In [63]:
(cont-frac-iter
  (lambda (i) 1.0)
  (lambda (i) 1.0)
  3)

depth: 3


0.6666666666666666

In [64]:
(display (format "~a\n" (cont-frac-iter (lambda (i) 1.0) (lambda (i) 1.0) 10)))
(display (format "~a\n" (cont-frac-iter (lambda (i) 1.0) (lambda (i) 1.0) 100)))

depth: 10
0.6179775280898876
depth: 100
0.6180339887498948


#### Exercise 1.38: approximate the natural logarithms based on Euler’s expansion.

- N_i = 1,1,1,1,1,...
- D_i = 1,2,1,1,4,1,1,6,1,1,8,...

  1, 2, 1, 1, 4, 1, 1, 6, 1, 1, 8, ...
  0, 1, 2, 3, 4, 5, 6, 7, 8, 9,10, ...
  ., ., ., 1, ., ., 2, ., ., 3,  ., ...
  ., 1, ., ., 2, ., ., 3, .  ., 4, ...

In [50]:
(define (print-loop f max n)
  (define (exec n)
    (display (format "~a\n" (f n)))
    (print-loop f max (+ n 1)))
  (if (< n max) (exec n)))

In [51]:
(define (d _i)
  (let ((i (+ _i 2)))
    (display (format "i: ~a, cond: ~a: res: " i (% i 3)))
    (if (> (% i 3) 0)
        1
        (* 2 (/ i 3)))))

In [52]:
(print-loop d 10 0)

i: 2, cond: 2: res: 1
i: 3, cond: 0: res: 2
i: 4, cond: 1: res: 1
i: 5, cond: 2: res: 1
i: 6, cond: 0: res: 4
i: 7, cond: 1: res: 1
i: 8, cond: 2: res: 1
i: 9, cond: 0: res: 6
i: 10, cond: 1: res: 1
i: 11, cond: 2: res: 1


#f

In [65]:
(cont-frac
  (lambda (i) 1.0)
  (lambda (i_) 
    (let ((i (+ i_ 1)))
      (if (> (% i 3) 0)
          1
          (* 2 (/ i 3)))))
  100)

depth: 100


0.7182818284590453

#### Exercise 1.39: A continued fraction representation of the tangent function

1,2,3,4,5...
2,4,6,8,10
1,3,4,7

In [83]:
(define (tan-cf x k)
  (cont-frac
    (lambda (i) 
      (if (= i 1) x (- (* x x))))
    (lambda (i) (- (* 2 i) 1))
    k))

In [84]:
(import "math")

(tan-cf (/ math.pi 3) 10)

depth: 10


1.7320508075688146

### 1.3.4 Procedures as Returned Values

In [2]:
(define (average x y) (/ (+ x y) 2))
(define (average-damp f)
  (lambda (x) (average x (f x))))

In [88]:
(define (square x) (* x x))

; average of 10 and 100 is 55
((average-damp square) 10)

55

In [90]:
(define (sqrt x)
  (fixed-point (average-damp (lambda (y) (/ x y)))
               1.0))

In [93]:
(sqrt 5)

guess: 1.0
guess: 3.0
guess: 2.3333333333333335
guess: 2.238095238095238
guess: 2.2360688956433634


2.2360688956433634

#### Newton’s method

square-root procedure -> special case of Newton's method.

- a differentiatiable function `x -> g(x)`
  - a solution of `g(x) = 0` is a fixed point of the function `x -> f(x)`
    - `f(x) = x - (g(x) / Dg(x))`
    - `Dg(x)` is the derivative of g evaluated at x
      - `Dg(x) = (g(x + dx) - g(x)) / dx`
      - in the limit of small `dx`

In [94]:
(define dx 0.00001)
(define (deriv g)
  (lambda (x) (/ (- (g (+ x dx)) (g x)) dx)))

In [96]:
((deriv (lambda (x) (* x x x))) 5)

75.00014999664018

In [97]:
; deriv of `x ^ 3` -> `3 * x ^ 2`
(* 3 (* 5 5))

75

In [98]:
(define (newton-transform g)
  (lambda (x) (- x (/ (g x) ((deriv g) x)))))

In [99]:
(define (newtons-method g guess)
  (fixed-point (newton-transform g) guess))

Find the value which satisfies`f(y) = 0` where `y -> y ^ 2 - x`.

In [100]:
(define (sqrt x)
  (newtons-method
    (lambda (y) (- (* y y) x)) 1.0))

In [101]:
(sqrt 5)

guess: 1.0
guess: 2.99999000003607
guess: 2.3333322222345143
guess: 2.238095396823653
guess: 2.2360689003143683


2.2360689003143683

### Abstractions and first-class procedures

In [102]:
(define (fixed-point-of-transform g transform guess)
  (fixed-point (transform g) guess))

Recast the first sqrt computation (y -> x / y)
```s
(define (sqrt x)
  (fixed-point-of-transform
    (lambda (y) (/ x y)) average-damp 1.0))
```

Recast the second sqrt computation (y -> y^2 - x)
```s
(define (sqrt x)
  (fixed-point-of-transform
    (lambda (y) (- (square y) x)) newton-transform 1.0))
```


first-class elements are:

- They may be named by variables.
- They may be passed as arguments to procedures.
- They may be returned as the results of procedures.
- ey may be included in data structures.


#### Exercise 1.40: cubic with newtons-method procedure

In [105]:
(define (cubic a b c)
  (lambda (x)
    (+ (* x x x) (* a x x) (* b x) c)))

In [109]:
; x^2 + x^2 + x + 1 = 0 -> x = -1
(newtons-method (cubic 1 1 1) 1.0)

guess: 1.0
guess: 0.33333777776275186
guess: -0.40739341574970156
guess: -1.4188731238603447
guess: -1.1184919351394478
guess: -1.0124818785025846
guess: -1.000153742427375
guess: -1.000000022096024


-1.000000022096024

#### Exercise 1.41: double 

In [110]:
(define (double f)
  (lambda (x) (f (f x))))

In [112]:
(define (inc x) (+ x 1))
((double inc) 1)

3

```s
(((double (double double)) inc) 5)
```

```s
(((double (double double)) f) x)
(((double double) ((double double) f)) x)
(((double double) (double (double f))) x)
(((double (double (double (double f))))) x)
```

```s
(((double (double (double (double (lambda (x) (+ 1 x))))))) x)
(((double (double (double (lambda (x) (+ 2 x)))))) 5)
(((double (double (lambda (x) (+ 4 x))))) 5)
(((double (lambda (x) (+ 8 x)))) 5)
((lambda (x) (+ 16 x)) 5)
(+ 16 5)
21
```

In [113]:
(((double (double double)) inc) 5)

21

#### Exercise 1.42: procedure to compose functions
- one-argument functions
  - f
  - g
- composition
  - x -> f(g(x))


In [118]:
(define (compose f g) (lambda (x) (f (g x))))

In [119]:
((compose square inc) 6)

49

#### Exercise 1.43: repeated application

In [174]:
(define (repeatd f n)
  (if (= n 1)
      f 
      ; (compose f (repeatd f (- n 1)))
      (lambda (x) (f ((repeatd f (- n 1)) x)))))

In [175]:
((repeatd (lambda (x) (+ x 1)) 3) 1)

4

```s
((repeatd square 2) 5)
```
square twice for 5 -> 5^4 = 625

In [176]:
((repeatd square 2) 5)

625

#### Exercise 1.44: smoothing a function
- f: function
- dx: some small number

- smoothed version of f
  => value at ponit x = average of `f(x - dx)`, `f(x)` and `f(x + dx)`

`(define (smmoth f) <smoothed-f>)`

In [195]:
(define (smooth f dx)
  (lambda (x) 
    (/ (+ (f (- x dx)) 
          (f x) 
          (f (+ x dx)))
        3)))

In [202]:
; https://billthelizard.blogspot.com/2010/08/sicp-144-smoothing-function.html
(display (math.sin (/ math.pi 2)))
(newline)
(display ((smooth math.sin 0.7) (/ math.pi 2)))

1.0
0.8432281248563256

In [207]:
(define (n-fold-smooth f dx n)
  (repeatd (smooth f dx) n))

In [209]:
(display ((n-fold-smooth math.sin 0.7 2) (/ math.pi 2)))
(newline)
(display ((n-fold-smooth math.sin 0.7 3) (/ math.pi 2)))
(newline)
(display ((n-fold-smooth math.sin 0.7 4) (/ math.pi 2)))

0.6297176112540722
0.4965910037020933
0.4017400886852076

#### Exercise 1.45: 


In [210]:
(define (cube-root x)
  (fixed-point (average-damp (lambda (y) (/ x (* y y))))
               1.0))

In [212]:
(cube-root 8)

guess: 1.0
guess: 4.5
guess: 2.447530864197531
guess: 1.8914996576441667
guess: 2.0637643832634476
guess: 1.9710425766479744
guess: 2.0151199754332096
guess: 1.992609760395472
guess: 2.0037362842809587
guess: 1.998142301706526
guess: 2.0009314406381735
guess: 1.9995349299633447
guess: 2.0002326972862416
guess: 1.9998836919616
guess: 2.0000581641656563
guess: 1.999970920454376
guess: 2.0000145404070393
guess: 1.9999927299550464
guess: 2.000003635062117


2.000003635062117

In [214]:
(cube-root 125)

guess: 1.0
guess: 63.0
guess: 31.51574703955656
guess: 15.82079874887776
guess: 8.160102055401849
guess: 5.018668937288702
guess: 4.990769572006499
guess: 5.004640837297728
guess: 4.997686034575535
guess: 5.0011585900348
guess: 4.999421107557479
guess: 5.00028954677172
guess: 4.999855251763398
guess: 5.000072380404159
guess: 4.999963811369566
guess: 5.000018094708105
guess: 4.999990952744172
guess: 5.0000045236524695


5.0000045236524695

In [215]:
(define (fourth-root x)
  (fixed-point (average-damp (lambda (y) (/ x (* y y y))))
               1.0))

In [ ]:
; (fourth-root 16) won't finish

In [6]:
(define (fourth-root x)
  (fixed-point (average-damp (average-damp (lambda (y) (/ x (* y y y)))))
               1.0))

In [8]:
; this procedure works because of double? average-damp
(fourth-root 16)

guess: 1.0
guess: 4.75
guess: 3.5998232249599065
guess: 2.7856139316659103
guess: 2.274263910561008
guess: 2.045743730517053
guess: 2.0015115314098866
guess: 2.000001711389449


2.000001711389449